# 5. Value Betting Calculator

In this notebook we'll make use of our match dataset and the Logistc Regression model we've built in the last stage, to predict the chances for each player to win in any given match.

We'll then compare these chances with the odds available for those matches, and build a Dataframe where each row will contain information regarding which player we would have to bet on!



trying to simulate bets on matches where we think to have an edge on the betting market (i.e. a value bet occurs when the real odds for an event are higher than the odds "predicted" by our model).

Finally, we'll compare several betting strategies and we'll find out if it's possible to make a good ROI backtesting our betting strategies.

Let's import the libraries needed for this notebook:

In [1]:
import pandas as pd
from math import exp
import pickle

Now, we'll load the Logistc Regression model trained in last notebook:

In [2]:
lreg = pickle.load(open("logistic_regression.lr", 'rb'))

Let's take a look at the coefficients and the intercept of this model:

In [3]:
lreg.coef_

array([[-7.38667203e-03,  1.06399179e-02,  8.41853267e-03,
        -4.02306203e-03, -2.89954052e-02,  2.78531804e-02,
         1.23497635e-02,  1.69745683e-02, -1.69951653e-02,
         4.06163471e-02, -4.96922677e-02,  0.00000000e+00,
        -6.24767193e-03,  1.52472500e-02, -7.32217665e-02,
         2.36171291e-02, -2.90847929e-02,  8.68967594e-02,
        -1.51170519e-02,  1.16339053e-03,  3.24765632e-01,
        -2.70326405e-01,  4.75992371e-01,  1.78567091e-01,
        -5.97096237e-01, -2.48846237e+00,  2.26352399e+00,
        -3.03947649e+00,  3.43256601e+00]])

In [4]:
lreg.intercept_

array([0.00433278])

Now, let's open the Coefficients dataframe saved in the last notebook, and let's create a dictionary object based on the dataframe, where the key is going to be the attribute name and the value will be the attribute's coefficient:

In [5]:
coefs_df = pd.read_csv("csv/Coefficients.csv")

In [6]:
coefs_df.head()

,Column,Coef
0,Indoor,-0.007387
1,Outdoor,0.010640
2,Carpet,0.008419
3,Clay,-0.004023
4,Grass,-0.028995


In [7]:
coefs_map = {}
for ix, row in coefs_df.iterrows():

    attribute = row['Column']
    attribute_coef = row['Coef']

    coefs_map[attribute] = attribute_coef

In [8]:
coefs_map

{'Indoor': -0.0073866720346099715,
 'Outdoor': 0.010639917903172487,
 'Carpet': 0.008418532672556997,
 'Clay': -0.004023062033578142,
 'Grass': -0.028995405210230762,
 'Hard': 0.027853180439818367,
 'ATP250': 0.01234976349121474,
 'ATP500': 0.016974568291650945,
 'Grand Slam': -0.016995165311491058,
 'Masters 1000': 0.04061634710986834,
 'Masters Cup': -0.049692267712689565,
 '0th Round': 0.0,
 '1st Round': -0.00624767192967942,
 '2nd Round': 0.015247250031749909,
 '3rd Round': -0.07322176649015735,
 '4th Round': 0.02361712906780745,
 'Quarterfinals': -0.02908479289950005,
 'Round Robin': 0.0868967594291554,
 'Semifinals': -0.015117051870574869,
 'The Final': 0.0011633905297499774,
 'Rank Index': 0.32476563162455485,
 'Pl0 Recent Form': -0.2703264053963188,
 'Pl0 Form': 0.4759923706696832,
 'Pl1 Recent Form': 0.1785670912193669,
 'Pl1 Form': -0.5970962366116814,
 'Pl0 Perf. vs Similar Opponent': -2.4884623654880484,
 'Pl1 Perf. vs Similar Opponent': 2.2635239881411184,
 'Pl0 Surface Pe

Let's make a quick analysis on the greatest indicators for player 1 to win a match against player 0.
Let's see which attributes matter the most:

In [19]:
coefs_df[abs(coefs_df['Coef']) > 0.1].sort_values('Coef', ascending = False)

,Column,Coef
28,Pl1 Surface Performance,3.432566
26,Pl1 Perf. vs Similar Opponent,2.263524
22,Pl0 Form,0.475992
20,Rank Index,0.324766
23,Pl1 Recent Form,0.178567
21,Pl0 Recent Form,-0.270326
24,Pl1 Form,-0.597096
25,Pl0 Perf. vs Similar Opponent,-2.488462
27,Pl0 Surface Performance,-3.039476


We can see that the most important features are:
- Player performance on specific surface
- Player performance against similar ranked opponents
- The opponent's form
- The rank index
- The players recent form

Let's load our matches Dataset

In [10]:
df = pd.read_csv("csv/FeatureCalculated_Data.csv")

Now our goal is to build another Dataframe, containing the informations about the the players, who wins each match, the average and maximum odds available for each player, and we'll use the logistic regression coefficients (and the intercept) to calculate the predicted chances to win for each player:

In [11]:
betting_data_df = df[(df["Avg_Pl0"] > 1) & (df["Avg_Pl1"] > 1) & (df["Max_Pl0"] > 1) & (df["Max_Pl1"] > 1)]

betting_df_list = []
for ix, irow in betting_data_df.iterrows():

    current_linear_combination = lreg.intercept_[0]
    for attribute in list(coefs_map.keys()):
        attr_value = irow[attribute]
        attr_weighted = attr_value * coefs_map.get(attribute)
        current_linear_combination += attr_weighted
    
    prob_p1_win = round(exp(current_linear_combination) / (1 + exp(current_linear_combination)), 2)
    prob_p0_win = round(1 - prob_p1_win, 2)

    betting_df_list.append(   {"Pl0":irow['Player 0'], "Pl1":irow['Player 1'], "Winner":irow['Won'], "Pl0 %":prob_p0_win, "Pl1 %":prob_p1_win, \
                              "Pl0 Avg odds": irow['Avg_Pl0'], "Pl1 Avg odds": irow['Avg_Pl1'], "Pl0 Max odds": irow['Max_Pl0'],
                               "Pl1 Max odds": irow['Max_Pl1']
                              })
    
betting_df = pd.DataFrame(betting_df_list)

In [12]:
betting_df.head()

,Pl0,Pl1,Winner,Pl0 %,Pl1 %,Pl0 Avg odds,Pl1 Avg odds,Pl0 Max odds,Pl1 Max odds
0,De Bakker T.,Falla A.,0.0,0.62,0.38,1.52,2.46,1.62,2.79
1,Starace P.,Hajek J.,1.0,0.67,0.33,1.34,3.12,1.40,3.64
2,Schwank E.,Fognini F.,0.0,0.35,0.65,1.99,1.77,2.14,1.85
3,Rochus C.,Garcia-Lopez G.,1.0,0.32,0.68,3.02,1.36,3.25,1.45
4,Bellucci T.,Nieminen J.,0.0,0.58,0.42,1.46,2.63,1.50,3.09


Let's define a helper funxtion that takes a change in input and returns the 'fair' odds for that chance value.

In [13]:
def right_odds(chance):
    return round(1/chance,2)

Now, we have everything we need to make decisions based on which player, for each match, we get the most value betting on!
In fact, next we're going to append to this betting dataframe another 4 columns.

The fair odds for both player 0 and player 1, based on the chances of each player to win according to the Logistic Regression model, and the value that we would get betting on each player at the max odds available!

In [14]:
for ix, row in betting_df.iterrows():
    right_odds_p0 = right_odds(row['Pl0 %'])
    right_odds_p1 = right_odds(row['Pl1 %'])
    bet_value_p0 = (row['Pl0 Max odds'] - right_odds_p0)/max(right_odds_p0,row['Pl0 Max odds'])
    bet_value_p1 = (row['Pl1 Max odds'] - right_odds_p1)/max(right_odds_p1,row['Pl1 Max odds'])
    betting_df.at[ix, 'Fair odds for Pl0'] = right_odds_p0
    betting_df.at[ix, 'Fair odds for Pl1'] = right_odds_p1
    betting_df.at[ix, 'Bet on Pl0 Value'] = bet_value_p0
    betting_df.at[ix, 'Bet on Pl1 Value'] = bet_value_p1

In [15]:
betting_df

,Pl0,Pl1,Winner,Pl0 %,Pl1 %,Pl0 Avg odds,Pl1 Avg odds,Pl0 Max odds,Pl1 Max odds,Fair odds for Pl0,Fair odds for Pl1,Bet on Pl0 Value,Bet on Pl1 Value
0,De Bakker T.,Falla A.,0.0,0.62,0.38,1.52,2.46,1.62,2.79,1.61,2.63,0.006173,0.057348
1,Starace P.,Hajek J.,1.0,0.67,0.33,1.34,3.12,1.40,3.64,1.49,3.03,-0.060403,0.167582
2,Schwank E.,Fognini F.,0.0,0.35,0.65,1.99,1.77,2.14,1.85,2.86,1.54,-0.251748,0.167568
3,Rochus C.,Garcia-Lopez G.,1.0,0.32,0.68,3.02,1.36,3.25,1.45,3.12,1.47,0.040000,-0.013605
4,Bellucci T.,Nieminen J.,0.0,0.58,0.42,1.46,2.63,1.50,3.09,1.72,2.38,-0.127907,0.229773
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17099,Zverev A.,Youzhny M.,0.0,0.56,0.44,1.60,2.30,1.66,2.45,1.79,2.27,-0.072626,0.073469
17100,Troicki V.,Wawrinka S.,1.0,0.18,0.82,4.34,1.21,4.91,1.24,5.56,1.22,-0.116906,0.016129
17101,Zverev A.,Berdych T.,0.0,0.17,0.83,2.68,1.46,3.12,1.57,5.88,1.20,-0.469388,0.235669
17102,Bautista Agut R.,Wawrinka S.,1.0,0.28,0.72,3.40,1.31,3.80,1.41,3.57,1.39,0.060526,0.014184


We just obtained 17104 matches where we can bet on the player who's value is higher!
Let's save this dataframe!

In [16]:
betting_df.to_csv("csv/Betting_Data.csv", index=False)